In [1]:
%pip install -U langchain langchain-community openai python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [4]:
from dotenv import dotenv_values

In [5]:
config = dotenv_values(".env")
openai_api_key=config['OPENAI_API_KEY'] 

In [6]:
from langchain_community.chat_models import ChatOpenAI,openai 
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

In [7]:
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough
from langchain_community.callbacks import get_openai_callback

In [8]:
text='হ্যালো এভারিওয়ান সালালুমুয়ালাইকুম আমরা আজকে মেশিন লার্নিং এবং অন্যান্য বিষয়ের উপর একটি বিস্তারিত আলোচনা করব এই সভাটি আমাদের প্রজেক্টের সম্পর্কে আরও পরিকল্পিতভাবে ধারণা দেওয়ার জন্য সময়ের সাথে আলোচনা করব সবাই আমাদের নোটবুক এবং পেন বের করি আমরা সবাই আশা করছি নোটবুক নিব আচ্ছা ঠিক আছে প্রথমে হলো আমরা আমাদের প্রজেক্টের লক্ষ্য এবং উদ্দেশ্য নির্ধারণ করতে চাই এই প্রজেক্টে আমরা কি ধরনের ডেটা ব্যবহার করব কি ধরনের মডেল তৈরি করব এবং অ্যাড দ্য এন্ড কনফ্লুশন কি হতে যাচ্ছে তা স্পষ্ট করতে হবে কারণ যেহেতু আমাদের প্রজেক্টের ডেললাইন খুবই ক্লোজ ঘনিয়ে আসছে ওকে ধরুন আমাদের প্রজেক্টের উদ্দেশ্য এবং লক্ষ্য নির্ধারণ হলো কিন্তু আসা যাক আমরা ডেটা কালেকশন এবং প্রিপ্রসেসিং এ এর দুটি ধাপ অনেক গুরুত্বপূর্ণ ডেটা সংগ্রহ এবং প্রিপ্রসেসিং কর আমাদের প্রথম ধাপ আমাদের যেহেতু আমাদের এই কাজটা খুবই দ্রুত সম্পন্ন করতে হবে আমরা আশা করছি এটা পাঁচ এপ্রিলের মধ্যেই আমরা করে ফেলব কিন্তু এখন আমরা আলোচনা করি কিভাবে এই ডেটা কালেকশন এবং প্রিপ্রসেসিং করা যায় প্রথমে আমরা চাচ্ছিলাম হচ্ছে হল ডেটা কালেকশন বিভিন্ন ওপেন সোর্স জায়গা থেকে কালেক্ট করার এই ওপেন সোর্স জায়গাতে আমরা বিভিন্ন ডেটা কালেক্ট করতে পারি যাই হোক ফার্স্ট এখন আমাদের নির্ধারণ করতে হবে প্রিপ্রসেসিংয়ের জন্য প্রিপ্রসেসিংয়ের জন্য আমরা প্রিভিয়ার্স কোনো প্রজেক্ট থেকে ধারণা নিতে পারি প্রিভিয়ার্স প্রজেক্টের ফর এক্সেম্পল আমাদের প্রজেক্ট নাম্বার এক ছিল প্রজেক্ট নাম্বার এক থেকে আমরা বিভিন্ন ধরনের কাজকর্ম করতে পারি প্রজেক্ট নাম্বার একে যেহেতু আমরা যে মডেলটি ইউজ করেছিলাম সেই মডেলটা আমরা ইউজ করতে পারি আচ্ছা ঠিক আছে আমাদের এখন মডেল তৈরি এবং এখন ধরা যাক আমরা ডেটা কালেকশন করেছি আচ্ছা আমাদের টাইমলান্ড এখন বিস্তারিত আলোচনা করি প্রথমত আমরা ডেটা কালেকশন করব তারপর প্রিপ্রসেসিং করব যেটা আমরা এপ্রিল পাঁচের মধ্যে করব এবং এখন এপ্রিল পাঁচের পরে আমরা প্রিপ্রসেসিং শুরু করে দেব প্রিপ্রসেসিং শুরু করলে তারপর আমরা প্রিপ্রসে এবং মডেল তৈরির পর্যায়ে আসব ওকে এই অংশটি বেশ চ্যালেঞ্জিং হতে পারে আমাদের প্রজেক্ট সম্পন্ন করার জন্য আমি আশা করছি দুই সপ্তাহের মধ্যে মডেল তৈরি করব এবং টেস্টিং করে ফেলব তাহলে আমরা এপ্রিলের শেষের দিকে আমরা পুরা প্রজেক্টটি দাঁড় করে ফেলব সো আমাদের হাতে মোটামুটি এক মাসের মতো সময় আছে এ এক মাসে আমরা মডেল তৈরি টেস্টিং সবকিছুই করব এখন আমরা যে যার মতো কাজ ভাগ করেনি আচ্ছা ঠিক আছে আপনাদের কোন প্রশ্ন থাকলে বা কোন স্কুয়ারি থাকলে আমাকে জানান এখন তাহলে এভাবে এখানে আমরা দলের মিটিং শেষ করছি'


In [9]:
qa_system_prompt="""Given the context of a meeting. You're task is follow the example format and wirte the 3 variables for the given context.
    User proper formatting
    Don't include input in answer
    
    Context: {context}
   
    
    You must return three variables, `Summary` , `Crucial Dealine` , `Follow-up actions`.
    
    
    Examples:
      
      Input: 'ওকে গুড মর্নিং এভ্রিওয়ান এন্ড ওয়েলকাম টু আর মিটিং রিগার্ডিং দ্য আকামিং আইসিটি ফেস্ট ইয়ার হিয়ার টু ডিসকাস দ্য ডিটেইলস এন্ড টাস্ক ফর অর্গানাইজিং বিস এক্সাইটিং ইভেন্ট লেটস গেট স্টার্টেড বাই রিভিউিং দ্য ইম্পটেন্ট ইনফরমো এই ইভেন্টের তারিখ সমূহ নিয়ে আলোচনা করব আইসিটি ফেস্ট কবে অনুষ্ঠিত হবে আমরা ডিসাইড করেছি যে আইসিটি ফেস্ট উইল টেক প্লেস অন এপ্রিল টোয়েন্টি সিক্স এন্ড এপ্রিল টোয়েন্টি সেভেন টোয়েন্টি টোয়েন্টি ফোর ওকেই এখন আমরা বাকি সমস্ত কার্যক্রম এবং দায়িত্বগুলি পর্যালোচনা করব প্রথমে আমরা অত্যাবশ্যক কাজগুলি কি কি হওয়া উচিত প্রথমে আমাদের ভেন্যু ফাইনালাইজ করতে হবে স্পন্সর অ্যারেঞ্জ করতে হবে আমাদের ডিফেন্ড ভেন্ডার্সদের সাথে কথাবার্তা বলতে হবে ইকুইপমেন্টস এন্ড ম্যাটেরিয়ালের জন্য এবং বিভিন্ন সোশ্যাল মিডিয়া চ্যানেল থেকে প্রোমোট করতে হবে ভেন্ডার ওকে এখন আমরা প্রত্যেকটা টাস্ক আরও ফার্দার ব্রেকডাউন করি কে কে রেসপন্সিবল হবে কার কোন কাজের জন্য ওকে রাকিনা আব্রার আপনি ভেন্যু বুকিংয়ের দায়িত্বে থাকবেন আপনার কাজ হচ্ছে হল পটেনশিয়াল লোকেশনসে রিচ আউট করবেন সবার সাথে কথা বলবেন এখন সবার সাথে কথা বলে আপনি ভেন্যু ঠিক করবেন আপনি আমাকে দশ দিনের মধ্যেই এই ইনফরমেশনটা আমাকে জানাবেন ওকে এবং স্পন্সরশিপের জন্য ইস্তেহাগ আপনি হচ্ছে পটেনশনিয়াল স্পন্সর এবং স্পন্সরশিপ প্রপোজাল ড্রাফট করবেন আপনার কাজ হচ্ছে হল আইসিটি ফেস্টের গুরুত্বপূর্ণ কেন আপনি কেন তাঁর স্পন্সরের সামনে তুলে ধরবেন আমিন উই নিট টু হাইলেট দ্য বেনিফিটিস অফ স্পন্সর ইন দ্য আইসিটি ফেস্ট ওকে এখন ইস্তিহাদ আপনার কাজে গেল এখন আমরা আসি হল রাফিয়া হাসান চৌধুরীর কাজে রাফিয়া হাসান আপনি হচ্ছে হলেন ডিফার্ন টাইমস অফ ভেন্ডরস এবং ফুড কিভাবে ইকুইপমেন্টস আছে এবং আর কি কি ধরনের ক্রেস্ট গিপস সবকিছু আছে সবকিছু আপনি হ্যান্ডেল করবেন যেহেতু আপনার ভেন্ডর ম্যানেজমেন্ট এক্সপিরিয়েন্স আছে আপনি এসব কাজ করবেন আপনি চাইলে ঢাকার বিভিন্ন এরিয়ায় যেতে পারেন যেমন কাঁটাবন এখন আপনার ডিফারেন্ড টাইভস অফ ইকুইপমেন্টস আর ম্যাটেরিয়ালস লাগলে আপনি আপনার আমাদের ফিনান্সিয়াল অফিসারের সাথে কথা বলবেন টাকা পয়সার ব্যাপারে এবং ভলেন্টিয়ার্স আপনি রাফিত হাসান আপনি ভলেন্টিয়ারের দায়িত্বে থাকবেন আপনি আমাদের যেকোনো ধরনের ভলেন্টিয়ার যারা আছেন তাদের সবাইকে ম্যানেজ করবেন এবং শেষে মৌদুল হাসান আপনি মার্কেটিং অ্যান্ড সোশ্যাল মিডিয়া আপনি ইউ ক্রিয়েট প্রোমোশনাল ম্যাটেরিয়ালস অ্যান্ড ম্যানেজ আওয়ার অনলাইন প্রেজেন্স টু অ্যাট্রাক দ্য পার্টি সিপিসিয়েন্যাংক ইউ ফর বিয়িং উইথাস নাও লেটস এন্ড শিওর আমরা এই মিটিংটা শেষ করছি এবং আমরা সবার সবার ডেডলাইন মিট করি এবং আশা করছি আমাদের আইসিটি ফেস্ট একটি গ্র্যান্ড সাকসেস হবে সবাই সমাপ্তি প্রশ্ন করছি'  
      Output: 'Summary:এই মিটিংটি আইসিটি ফেস্ট আয়োজনের তারিখ এবং ব্যবস্থাপনা নিয়ে আলোচনা করে। ফেস্টটি এপ্রিল ২৬ থেকে ২৭ এপ্রিল অনুষ্ঠিত হবে। ভেন্যু ফাইনালাইজ করা, স্পন্সর অ্যারেঞ্জ করা, ডিফেন্ড ভেন্ডার্স এবং সোশ্যাল মিডিয়া চ্যানেল থেকে প্রোমোট করা সহ বিভিন্ন কাজ এক্সিকিউট করতে হবে।\nCrucial Dealine:ফেস্টটি এপ্রিল ২৬ থেকে ২৭ এপ্রিল অনুষ্ঠিত হবে\nFollow-up actions:ভেন্যু বুকিং নিশ্চিত করা-রাকিন আব্রার,স্পন্সরশিপ প্রপোজাল ড্রাফট করা-ইস্তিহাদ,ভলেন্টিয়ার ম্যানেজমেন্ট-রাফিয়া হাসান,মার্কেটিং এবং সোশ্যাল মিডিয়া প্রোমোশনাল ম্যাটেরিয়ালস তৈরি-মৌদুল হাসান'
      
      Input: 'গেম ডেভেলপমেন্ট নিয়ে একটি বৈঠক আয়োজন করা হচ্ছে যেখানে আলোচনা করা হবে গেম ডেভেলপমেন্ট প্রক্রিয়া, গেম প্রজেক্ট ম্যানেজমেন্ট, টেকনোলজি এবং প্ল্যাটফর্ম, মার্কেটিং পরিকল্পনা এবং আরও অনেক বিষয়ে। এই বৈঠকের মাধ্যমে গেম উন্নতির সমস্যা এবং সমাধান, বিভিন্ন প্রোজেক্টের অগ্রগতি, এবং আগামীর পরিকল্পনা নিয়ে আলোচনা করা হবে।গেম ডেভেলপমেন্ট প্রক্রিয়া এবং প্রজেক্টের অবলম্বনের জন্য শেষ তারিখঃ মার্চ ৩১, ২০২৪।প্রথমে প্রজেক্টের মূল ফীচার এবং কার্যপ্রণালী পর্যালোচনা করা হবে।প্রোজেক্টের অগ্রগতি নিয়ে নিয়মিতভাবে সম্পর্কিত দলের সাথে যোগাযোগ বজায় রাখা যাবে।মার্কেটিং পরিকল্পনা পর্যালোচনা করে গেমটির প্রচার-প্রচার নির্দেশ করা হবে।যেকোনো সমস্যার সামাধানের জন্য দ্রুত পদক্ষেপ নেওয়া হবে।'
      Output: 'Summary:একটি গেম ডেভেলপমেন্ট বৈঠকে গেম ডেভেলপমেন্ট প্রক্রিয়া, প্রজেক্ট ম্যানেজমেন্ট, টেকনোলজি এবং প্ল্যাটফর্ম, মার্কেটিং পরিকল্পনা এবং অন্যান্য বিষয়ে আলোচনা হচ্ছে। মার্চ ৩১, ২০২৪ এ প্রজেক্টের ডেডলাইন নির্ধারণ করা হয়েছে। বৈঠকে প্রথমে প্রজেক্টের মূল ফীচার এবং কার্যপ্রণালী নিয়ে আলোচনা করা হবে। প্রজেক্টের অগ্রগতি নিয়ে নিয়মিতভাবে সম্পর্কিত দলের সাথে যোগাযোগ রক্ষা করা হবে। মার্কেটিং পরিকল্পনা নিয়ে গেমটির প্রচার-প্রচার নির্দেশ করা হবে। যেকোনো সমস্যার সামাধানের জন্য দ্রুত পদক্ষেপ নেওয়া হবে।\nCrucial Dealine:গেম ডেভেলপমেন্ট প্রক্রিয়া এবং প্রজেক্টের অবলম্বনের জন্য শেষ তারিখঃ মার্চ ৩১, ২০২৪।\nFollow-up actions:প্রথমে প্রজেক্টের মূল ফীচার এবং কার্যপ্রণালী পর্যালোচনা করা হবে।প্রোজেক্টের অগ্রগতি নিয়ে নিয়মিতভাবে সম্পর্কিত দলের সাথে যোগাযোগ বজায় রাখা যাবে।মার্কেটিং পরিকল্পনা পর্যালোচনা করে গেমটির প্রচার-প্রচার নির্দেশ করা হবে।যেকোনো সমস্যার সামাধানের জন্য দ্রুত পদক্ষেপ নেওয়া হবে।'
      
    """


In [10]:
prompt = ChatPromptTemplate.from_messages(
    [("system", qa_system_prompt)]
)


In [11]:
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo-0125",
    streaming=True,
    temperature=0.5, 
    api_key=  openai_api_key
)

e:\IUT_FRISBEE\env\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [12]:
chain= create_stuff_documents_chain(llm, prompt)

In [13]:
docs=[Document(page_content=text)]



In [14]:
docs

[Document(page_content='হ্যালো এভারিওয়ান সালালুমুয়ালাইকুম আমরা আজকে মেশিন লার্নিং এবং অন্যান্য বিষয়ের উপর একটি বিস্তারিত আলোচনা করব এই সভাটি আমাদের প্রজেক্টের সম্পর্কে আরও পরিকল্পিতভাবে ধারণা দেওয়ার জন্য সময়ের সাথে আলোচনা করব সবাই আমাদের নোটবুক এবং পেন বের করি আমরা সবাই আশা করছি নোটবুক নিব আচ্ছা ঠিক আছে প্রথমে হলো আমরা আমাদের প্রজেক্টের লক্ষ্য এবং উদ্দেশ্য নির্ধারণ করতে চাই এই প্রজেক্টে আমরা কি ধরনের ডেটা ব্যবহার করব কি ধরনের মডেল তৈরি করব এবং অ্যাড দ্য এন্ড কনফ্লুশন কি হতে যাচ্ছে তা স্পষ্ট করতে হবে কারণ যেহেতু আমাদের প্রজেক্টের ডেললাইন খুবই ক্লোজ ঘনিয়ে আসছে ওকে ধরুন আমাদের প্রজেক্টের উদ্দেশ্য এবং লক্ষ্য নির্ধারণ হলো কিন্তু আসা যাক আমরা ডেটা কালেকশন এবং প্রিপ্রসেসিং এ এর দুটি ধাপ অনেক গুরুত্বপূর্ণ ডেটা সংগ্রহ এবং প্রিপ্রসেসিং কর আমাদের প্রথম ধাপ আমাদের যেহেতু আমাদের এই কাজটা খুবই দ্রুত সম্পন্ন করতে হবে আমরা আশা করছি এটা পাঁচ এপ্রিলের মধ্যেই আমরা করে ফেলব কিন্তু এখন আমরা আলোচনা করি কিভাবে এই ডেটা কালেকশন এবং প্রিপ্রসেসিং করা যায় প্রথমে আমরা চাচ্ছিলাম হচ্ছে হল ডেটা কালেকশন বিভিন্ন ওপেন স

In [19]:
chain.invoke({"context": docs})

'    Summary: এই সভাটি মেশিন লার্নিং এবং অন্যান্য বিষয়ের উপর বিস্তারিত আলোচনা করবে এবং প্রজেক্টের লক্ষ্য এবং উদ্দেশ্য নির্ধারণ করব।\nCrucial Deadline: প্রজেক্টের ডেডলাইন হলো পাঁচ এপ্রিল\nFollow-up actions: ডেটা কালেকশন এবং প্রিপ্রসেসিং করে প্রজেক্ট সম্পন্ন করা, মডেল তৈরি এবং টেস্টিং করা।'

In [18]:
for chunk in chain.stream({"context": docs}):
    print(chunk)
  


Summary
:
 এ
ই
 স
ভ
াট
ি
 ম
ে
শ
িন
 ল
ার
্ন
িং
 এ
ব
ং
 অ
ন
্য
ান
্য
 ব
িষ
য়
ে
 আ
ল
ো
চ
ন
া
 ক
র
ে
 প
্র
জ
ে
ক
্ট
 এ
র
 স
ম
্প
র
্ক
ে
 ধ
ার
ণ
া
 দ
ে
ও
য়
া
 হ
ব
ে
।
 ড
ে
ট
া
 ক
াল
ে
ক
শ
ন
 এ
ব
ং
 প
্র
িপ
্র
স
ে
স
িং
 প
্র
থ
ম
 ধ
াপ
।


Cr
uc
ial
 Deal
ine
:
 প
্র
জ
ে
ক
্ট
ে
র
 ড
ে
ড
ল
াই
ন
 হ
ল
ো
 প
াঁ
চ
 এ
প
্র
িল
।


Follow
-up
 actions
:
 ড
ে
ট
া
 ক
াল
ে
ক
শ
ন
 এ
ব
ং
 প
্র
িপ
্র
স
ে
স
িং
 প
্র
থ
ম
 ধ
াপ
,
 প
্র
জ
ে
ক
্ট
 ম
ড
ে
ল
 ত
ৈ
র
ি
 এ
ব
ং
 ট
ে
স
্ট
িং
।

